#### Importanto NLTK e RSLP:

In [140]:
!pip install emoji
!pip install nltk
import nltk 
from nltk.stem import RSLPStemmer
from emoji import UNICODE_EMOJI
import emoji
import re 
import functools
import operator
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
#nltk.download('rslp')
#nltk.download('stopwords')
#nltk.download('punkt')

### Criando funções necessárias para análise:

<h4>Função Tokenize:</h4><p>Essa função serve para quebrarmos nosso texto por palavras, criando desse modo um array com todas as palavras contidas dentro do texto.</p>

In [141]:
def Tokenize(sentence):
    sentence = sentence.lower()
    sentence = nltk.word_tokenize(sentence)
    return sentence

In [142]:
frase_teste = "Se ele falhou meu amigo, o problema é dele! Estou viajando nesse comentário."
frase_teste = Tokenize(frase_teste)
print(frase_teste)


['se', 'ele', 'falhou', 'meu', 'amigo', ',', 'o', 'problema', 'é', 'dele', '!', 'estou', 'viajando', 'nesse', 'comentário', '.']


<h4>Função Stemming:</h4><p>Essa função serve para diminuirmos a palavra até a sua raiz, pois assim, conseguimos tratar as palavras originais e suas respectivas derivações de uma mesma maneira.</p>

In [143]:
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

In [144]:
frase_teste = Stemming(frase_teste)
print(frase_teste)

['se', 'ele', 'falh', 'meu', 'amig', ',', 'o', 'problem', 'é', 'del', '!', 'est', 'viaj', 'ness', 'coment', '.']


<h4>RemoveStopWords:</h4><p>Essa função serve para retirarmos dentro do nosso array algumas palavras que não são interessantes para contabilizarmos uma pontuação na hora de classificar o nosso texto, então mantemos somente as palavras principais.</p>

In [145]:
def RemoveStopWords(sentence):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    phrase = []
    for word in sentence:
        if word not in stopwords:
            phrase.append(word)
    return phrase

In [146]:
frase_teste = RemoveStopWords(frase_teste)
print(frase_teste)

['falh', 'amig', ',', 'problem', 'del', '!', 'est', 'viaj', 'ness', 'coment', '.']


Palavras consideradas como stopwords:

In [147]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [ ]:
def space_cleaning(text):
    punctuation = '[!\-.:?;,''"@/]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    split_emoji = emoji.get_emoji_regexp().split(text_subbed)
    split_whitespace = [substr.split() for substr in split_emoji]
    split = functools.reduce(operator.concat,split_whitespace)
    return split

In [3]:
punctuation = "[!\-.:?;,''"@/]"
print(punctuation)

SyntaxError: invalid syntax (<ipython-input-3-e503c18acb99>, line 1)

In [150]:
def Train():
    training_data = []
    training_data.append({"classe":"amor", "frase":"Eu te amo"})
    training_data.append({"classe":"amor", "frase":"Você é o amor da minha vida"})
    training_data.append({"classe":"medo", "frase":"estou com medo"})
    training_data.append({"classe":"medo", "frase":"tenho medo de fantasma"})
    print("%s frases incluidas" % len(training_data))
    return training_data

In [151]:
Dados = Train()

4 frases incluidas


In [152]:
def Learning(training_data):
    corpus_words = {}
    for data in training_data:
        frase = data['frase']
        frase = Tokenize(frase)
        frase = Stemming(frase)
        frase = RemoveStopWords(frase)
        class_name = data['classe']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

In [153]:
def Learning(training_data):
    corpus_words = {}
    for data in training_data:
        frase = data['frase']
        frase = Tokenize(frase)
        frase = Stemming(frase)
        frase = RemoveStopWords(frase)
        class_name = data['classe']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in frase:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1
    return corpus_words

In [156]:
dados = Learning(Dados)
print(dados)

{'amor': {'amo': 1, 'voc': 1, 'am': 1, 'minh': 1, 'vid': 1}, 'medo': {'est': 1, 'med': 2, 'tenh': 1, 'fantasm': 1}}


In [162]:
def calculate_class_score(sentence,class_name):
    score = 0 
    sentence = Tokenize(sentence)
    sentence = Stemming(sentence)
    for word in sentence:
        if word in dados[class_name]:
            score += dados[class_name][word]
    return score

In [163]:
score = calculate_class_score("Tenho medo de baratas", "medo")
print(str(score))

3


In [164]:
def calculate_score(sentence):
    high_score = 0
    classname = 'default'
    for classe in dados.keys():
        pontos = 0
        pontos = calculate_class_score(sentence,classe)
        if pontos > high_score:
            high_score = pontos
            classname = classe
    return classname,high_score

In [166]:
print(calculate_score("Eu amo aquela casa"))

('amor', 1)


*Train.py

In [167]:
import nltk 
import re
import yaml
from nltk.stem import RSLPStemmer

stemmer = RSLPStemmer()
expression = '[!-@[-`{-¿ÆÐÑ×ØÝ-ßä-æëðñö-øý-ÿ]'

#Carrega o Corpus Words
def LoadMemory():
    fileW = open("words.nlp", 'r')
    words = fileW.read()
    fileW.close()
    words = yaml.load(words)
    return words

#Carrega as frases que foram treinadas
def LoadExamples():
    fileE = open("examples.nlp", 'r')
    examples = fileE.read()
    fileE.close()
    return examples

#Salva a corpus words
def SaveMemory(w):
    fileW = open("words.nlp", 'w')
    fileW.write(str(w))
    fileW.close()

#Salva as novas frases treinadas
def SaveExample(example):
    fileE = open("examples.nlp", 'a')
    fileE.write(example + "\n")
    fileE.close()

#Massa de dados para exemplo
def Examples():
    training_data = []
    training_data.append({"class":"saudade", "sentence":"sinto sua falta"})
    training_data.append({"class":"saudade", "sentence":"estou com saudades"})

    training_data.append({"class":"fome", "sentence":"estou com fome"})
    training_data.append({"class":"fome", "sentence":"to faminto"})

    training_data.append({"class":"medo", "sentence":"to com medo"})
    training_data.append({"class":"medo", "sentence":"tomei um susto"})

    Learning(training_data)

#Função responsavel por treinar a frase
def Learning(training_data):
    corpus_words = LoadMemory()

    for data in training_data:
        examples = LoadExamples()
        sentence = data['sentence']
        sentence = re.sub(expression, '', sentence)
        sentence = stemmer.stem(sentence.lower())


        if sentence in examples:
            continue
        
        SaveExample(sentence)
        sentence = nltk.word_tokenize(sentence)
        class_name = data['class']
        if class_name not in list(corpus_words.keys()):
            corpus_words[class_name] = {}
        for word in sentence:
            if word not in list(corpus_words[class_name].keys()):
                corpus_words[class_name][word] = 1
            else:
                corpus_words[class_name][word] += 1

    
    SaveMemory(corpus_words)




*Classification.py

In [169]:
import nltk 
import re
import yaml
from nltk.stem import RSLPStemmer

expression = '[!-@[-`{-¿ÆÐÑ×ØÝ-ßä-æëðñö-øý-ÿ]'
stemmer = RSLPStemmer()


#Carrega o Corpus Words
def LoadMemory():
    fileW = open("words.nlp", 'r')
    words = fileW.read()
    words = yaml.load(words)
    return words

#Função responsavel por calcular a pontuação por classe
def calculate_class_score(sentence,class_name):
    score = 0
    sentence = re.sub(expression, '', sentence)
    sentence = nltk.word_tokenize(sentence)
    for word in sentence:
        if stemmer.stem(word.lower()) in corpus_words[class_name]:
            score += corpus_words[class_name][stemmer.stem(word.lower())]
    return score

#Função responsavel por classificar a frase
def classifique(sentence):
    high_class = None
    high_score = 0
    for c in list(corpus_words.keys()):
        score = calculate_class_score(sentence, c)
        if score > high_score:
            high_class = c
            high_score = score

    print(str(high_class))
    return high_class

memory = LoadMemory()
corpus_words = memory

FileNotFoundError: [Errno 2] No such file or directory: ''